# Post Processing

This notebook demonstrates a full **post-processing pipeline** for point clouds, generating structured statistics and visualizations.

---

## Pipeline Overview

The workflow is divided into **11 main steps**:

1. **Extract archives** → Uncompress all submissions from `raw_dir` to `extracted_dir`.
2. **Analyze submission directory** → Scan `extracted_dir` to collect mandatory files:
   - `*_dense_pointcloud.las/laz`
   - `*_sparse_pointcloud.las/laz`
   - `*_extrinsics.csv`
   - `*_intrinsics.csv`
3. **Create point cloud symlinks** → Organize dense point cloud files by `(site, dataset)` using symbolic links in `processing_dir`.
4. **Convert point clouds to DEMs** → Generate DEMs from point clouds via PDAL; skip existing DEMs unless `overwrite=True`.
5. **Coregister DEMs** → Align all DEMs to their reference DEMs; skip existing coregistered DEMs if `overwrite=False`.
6. **Generate differential DEMs (DDEMs)** → Compute `ddem_before` and `ddem_after` for each DEM relative to the reference.
7. **Compute statistics** → Compute global statistics, coregistration shifts, point cloud metadata, and apply NMAD-based inliers filtering.
8. **Compute landcover-based statistics** → Stratify statistics for coregistered DEMs by reference landcover.
9. **Generate standard deviation DEMs (std DEMs)** → Compute std DEMs for raw and coregistered DEMs; process both all and inliers-only sets.
10. **Compute landcover statistics on std DEMs** → Stratify std DEM statistics by landcover and save results.
11. **Generate post-processing plots** → Create comprehensive visual summaries including:
    - Global and per-dataset summaries
    - DEM quality metrics and NMAD before/after coregistration
    - Coregistration shifts and landcover-based statistics
    - Slope and hillshade mosaics


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import history.postprocessing.pipeline as pp
import history.postprocessing_prefect.flows as ppfl
from prefect.task_runners import ConcurrentTaskRunner

## ⚙️ Settings and Path Management

Main directories used in the post-processing pipeline:

- **`raw_dir`** → Contains all raw input archives.  
- **`extracted_dir`** → Stores extracted archives.  
- **`processing_dir`** → Main workspace for intermediate and processed data.  
- **`plots_dir`** → Output folder for generated plots and visualizations.

> 💡 These directories are automatically created when running the Prefect flows.


In [ ]:
# paths
raw_dir = "/mnt/summer/USERS/DEHECQA/history/output/raw"
extracted_dir = "/mnt/summer/USERS/DEHECQA/history/output/extracted"
processing_dir = "/mnt/summer/USERS/DEHECQA/history/output/processing" 
plots_dir = "/mnt/summer/USERS/DEHECQA/history/output/plots" 

# other settings
OVERWRITE = False
DRY_RUN = False # set this to True to avoid process
PDAL_EXEC_PATH = "/home/godinlu/micromamba/envs/pdal/bin/pdal"
MAX_WORKERS = 4

### 🧩 Step 1 — Uncompress All Submissions

This step extracts all supported archive submissions from the `raw_dir` into the `extracted_dir`.  
The flow `uncompress_all_submissions` scans the input directory for archives (`.zip`, `.7z`, `.tgz`, `.tar.gz`, `.tar.bz2`, `.tar.xz`) and extracts each one into a corresponding folder in the output directory.

Extraction tasks run in parallel via Prefect, and existing folders are skipped unless `overwrite=True` is specified.  
Each archive produces one folder in `extracted_dir` containing the uncompressed contents of that submission.


In [ ]:
# task_runner = ConcurrentTaskRunner(max_workers=4)
# ppfl.uncompress_all_submissions.with_options(task_runner=task_runner)(raw_dir, extracted_dir, OVERWRITE)

pp.uncompress_all_submissions(raw_dir, extracted_dir, OVERWRITE)

### 🧩 Step 2 — Analyze Submission Directory And link files



In [ ]:
pp.index_submissions_and_link_files(extracted_dir, processing_dir)

### 🧩 Step 3 — Convert Point Clouds to DEMs

In this step, the flow `process_pointclouds_to_dems` converts all point clouds in the `processing_dir` into **Digital Elevation Models (DEMs)** for each site and dataset.

For each subdirectory, the flow:

- Retrieves the reference DEM for the dataset.
- Converts each point cloud into a DEM using the **PDAL** pipeline.
- Skips conversion if the DEM already exists and `overwrite=False`.
- Executes tasks asynchronously via Prefect, allowing parallel processing.
- Logs any errors encountered per file or dataset without stopping the overall execution.

All generated DEMs are saved under their corresponding site and dataset subdirectories in `processing_dir`.  
The `dry_run` option can be used to simulate the execution without creating any files.


In [ ]:
#task_runner = ConcurrentTaskRunner(max_workers=4)
#ppfl.process_pointclouds_to_dems.with_options(task_runner=task_runner)(processing_dir, PDAL_EXEC_PATH, OVERWRITE, DRY_RUN)

pp.process_pointclouds_to_dems(processing_dir, PDAL_EXEC_PATH, OVERWRITE, DRY_RUN)

### 🧩 Step 4 — Coregister DEMs

This step aligns all generated DEMs in the `processing_dir` to their corresponding **reference DEMs**.

The flow `process_coregister_dems` performs the following for each site and dataset:

- Retrieves the reference DEM and its mask.
- Aligns all raw DEMs to the reference using the coregistration pipeline.
- Skips DEMs that are already coregistered if `overwrite=False`.
- Submits each coregistration task asynchronously via Prefect for parallel execution.
- Logs any errors for individual DEMs or datasets without stopping the overall workflow.

All coregistered DEMs are saved under their respective site and dataset subdirectories in `processing_dir`.


In [ ]:
# task_runner = ConcurrentTaskRunner(max_workers=4)
# ppfl.process_coregister_dems.with_options(task_runner=task_runner)(processing_dir, OVERWRITE)

pp.process_coregister_dems(processing_dir, OVERWRITE)

### 🧩 Step 5 — Generate Differential DEMs (DDEMs)

In this step, the flow `process_generate_ddems` computes **differential DEMs (DDEMs)** for each dataset in the `processing_dir`.

For each site and dataset, the flow:

- Computes DDEMs by differencing each DEM against its corresponding reference DEM.
- Produces two sets of DDEMs:
  - `ddem_before` → from raw DEMs before coregistration.
  - `ddem_after` → from coregistered DEMs after alignment.
- Executes tasks asynchronously via Prefect for parallel processing.
- Skips existing DDEMs if `overwrite=False`.
- Logs any errors per file or dataset without interrupting the overall workflow.

All generated DDEMs are stored under their respective site and dataset subdirectories in `processing_dir`.


In [ ]:
# task_runner = ConcurrentTaskRunner(max_workers=4)
# ppfl.process_generate_ddems.with_options(task_runner=task_runner)(processing_dir, OVERWRITE)

pp.process_generate_ddems(processing_dir, OVERWRITE)

### 🧩 Step 6 — Compute Statistics

In this step, the flow `process_compute_statistics` computes **comprehensive statistics** for all DEMs, DDEMs, and point clouds in the `processing_dir`.

For each site and dataset, the flow:

1. Computes raster statistics for raw DEMs, coregistered DEMs, and DDEMs, either by retrieving existing results or submitting asynchronous tasks.
2. Extracts metadata from all point cloud files.
3. Computes coregistration shifts for coregistered DEMs.
4. Applies an **inliers filter** based on the NMAD of DDEMs using the specified `nmad_multiplier`.
5. Saves the resulting statistics as a CSV file within the corresponding subdirectory.

All tasks are executed asynchronously via Prefect. Errors for individual computations are logged without interrupting the overall workflow.


In [ ]:
# task_runner = ConcurrentTaskRunner(max_workers=4)
# ppfl.process_compute_statistics.with_options(task_runner=task_runner)(processing_dir)

pp.process_compute_statistics(processing_dir)

### 🧩 Step 7 — Compute Landcover-Based Statistics

In this step, the flow `process_compute_landcover_statistics` computes **landcover-stratified statistics** for all coregistered DEMs in the `processing_dir`.

For each site and dataset, the flow:

1. Retrieves the reference landcover map for the dataset.
2. Computes raster statistics for each coregistered DEM, stratified by landcover. Existing statistics are reused if available; otherwise, asynchronous tasks are submitted.
3. Aggregates and flattens the results into a single DataFrame per dataset.
4. Saves the resulting landcover statistics as a CSV file in the corresponding subdirectory.

All computation tasks are executed asynchronously via Prefect. Errors for individual DEMs or datasets are logged without stopping the overall workflow.


In [ ]:
# task_runner = ConcurrentTaskRunner(max_workers=4)
# ppfl.process_compute_landcover_statistics.with_options(task_runner=task_runner)(processing_dir)

pp.process_compute_landcover_statistics(processing_dir)

### 🧩 Step 8 — Generate Standard Deviation DEMs (std DEMs)

In this step, the flow `process_generate_std_dems` computes **standard deviation DEMs** for both raw and coregistered DEMs in the `processing_dir`.

For each site and dataset, the flow:

1. Retrieves the statistics DataFrame to determine available DEMs.
2. Prepares two sets for each DEM type: all DEMs and inliers-only DEMs.
3. Computes standard deviation DEMs for each set, skipping sets with insufficient DEMs or if outputs already exist and `overwrite=False`.
4. Submits asynchronous tasks to generate the std DEMs, logging progress via Prefect.

Warnings are issued for sets with insufficient DEMs, and errors in individual tasks are logged without stopping the overall workflow.  
All generated std DEMs are saved under the corresponding subdirectory's `std_dems_dir`.


In [ ]:
# task_runner = ConcurrentTaskRunner(max_workers=4)
# ppfl.process_generate_std_dems.with_options(task_runner=task_runner)(processing_dir, OVERWRITE)

pp.process_generate_std_dems(processing_dir, OVERWRITE)

### 🧩 Step 9 — Compute Landcover-Based Statistics on Standard Deviation DEMs

In this step, the flow `process_compute_landcover_statistics_on_std_dems` computes **landcover-stratified statistics** for standard deviation DEMs (std DEMs) in the `processing_dir`.

For each site and dataset, the flow:

1. Retrieves the reference landcover map for the dataset.
2. For both raw and coregistered DEMs, and for each subset ('all' and 'inliers'), computes raster statistics stratified by landcover. Existing statistics are reused when available; otherwise, asynchronous tasks are submitted.
3. Aggregates and flattens the results into a single DataFrame per dataset.
4. Saves the resulting std landcover statistics as a CSV file in the corresponding subdirectory.

All tasks and errors are logged through Prefect. Warnings are issued if no std DEMs are available or if computations fail, without stopping the overall workflow.


In [ ]:
# task_runner = ConcurrentTaskRunner(max_workers=4)
# ppfl.process_compute_landcover_statistics_on_std_dems.with_options(task_runner=task_runner)(processing_dir)

pp.process_compute_landcover_statistics_on_std_dems(processing_dir)

### 🧩 Step 10 — Generate Post-Processing Plots

In this final step, the flow `generate_postprocessing_plots` creates **visual summaries and plots** for all processed DEMs, DDEMs, and point cloud analyses.

For each site and dataset, the flow:

- Consolidates statistics and visualization tasks from the `processing_dir`.
- Generates a comprehensive set of plots, including:
  - Global summaries and per-dataset comparisons.
  - DEM quality metrics and NMAD before/after coregistration.
  - Coregistration shifts and landcover-based statistics.
  - Slope and hillshade mosaics for spatial interpretation.
- Saves all plots and visual summaries in the specified `plots_dir`.

This step provides a complete visual overview of the post-processing results for analysis and reporting purposes.


In [ ]:
task_runner = ConcurrentTaskRunner(max_workers=4)
ppfl.generate_postprocessing_plots.with_options(task_runner=task_runner)(processing_dir, plots_dir)

#pp.generate_postprocessing_plots(processing_dir, plots_dir, MAX_WORKERS)

## Visualize processing files

In [ ]:
processing_dir = pp.ProcessingDirectory(processing_dir)
processing_dir.submissions_summary()
processing_dir.plot()
